### Experiments with the AlphaVantage API

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
load_dotenv()

True

In [3]:
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

In [4]:
# Crypto exchange rates could be a fun project (or an extension to this project) too!
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=BTC&to_currency=EUR&apikey={ALPHA_VANTAGE_API_KEY}'
# r = requests.get(url)
# data = r.json()

# print(data)

### Fetch 100 ticker data points at a specified interval (in minutes; takes only 1, 5, 15, 30 and 60)

In [5]:
def fetch_ticker_data(symbol: str, interval: int, apiKey: str) -> pd.DataFrame:
    base_url = f'https://www.alphavantage.co/query'
    interval = str(interval) + "min"
    params = {
        "function": "TIME_SERIES_INTRADAY",
        "symbol": symbol,
        "interval": interval,
        "apikey": apiKey,
        "outputsize": "compact",
    }
    try:
        response = requests.get(base_url, params=params)
        data = response.json()[f"Time Series ({interval})"]
        df = pd.DataFrame(data)
        df = df.transpose()
        df = df.reset_index().rename(columns={"index": "timestamp", "1. open": "open", "2. high": "high", "3. low": "low", "4. close": "close", "5. volume": "volume"})
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df
    except Exception as e:
        print(e)
        return None

In [6]:
df = fetch_ticker_data(symbol="IBM", interval=5, apiKey=ALPHA_VANTAGE_API_KEY)

In [7]:
df

,timestamp,open,high,low,close,volume
0,2025-01-24 19:55:00,224.7800,224.7800,224.7800,224.7800,5
1,2025-01-24 19:45:00,224.6200,224.6200,224.6200,224.6200,4
2,2025-01-24 19:40:00,224.6000,224.6000,224.6000,224.6000,4
3,2025-01-24 19:30:00,224.8000,224.8000,224.8000,224.8000,7
4,2025-01-24 19:25:00,224.6200,224.8000,224.6200,224.7900,12
...,...,...,...,...,...,...
95,2025-01-24 11:00:00,226.3500,226.8104,226.3112,226.5400,19968
96,2025-01-24 10:55:00,226.4250,226.5100,226.3600,226.4100,9072
97,2025-01-24 10:50:00,226.3750,226.6300,226.3000,226.4795,22020
98,2025-01-24 10:45:00,226.0750,226.3800,226.0750,226.3750,169155


### Filter by time

In [8]:
# # est_time = datetime(2024, 1, 23, 14, 5, 0).replace(minute=0, second=0, microsecond=0) + timedelta(hours=3)
# est_time = datetime.now().replace(minute=0, second=0, microsecond=0) + timedelta(hours=3)
# print(type(est_time))
# print(est_time)

In [9]:
# df[df["timestamp"]==est_time]

In [10]:
def filter_stock_data(df: pd.DataFrame) -> pd.DataFrame:
    now = datetime.now()
    minutes = now.minute
    round_down_minutes = minutes // 5 * 5
    pst_time = datetime.now().replace(minute=round_down_minutes, second=0, microsecond=0) + timedelta(hours=3) # Rounds down time to the closest 5n minutes
    return_df = df[df["timestamp"]==pst_time]
    if return_df.empty:
        return None
    return return_df

In [11]:
print(filter_stock_data(df))

None


### Experiments with yfinance API

In [12]:
import yfinance as yf

In [13]:
data = yf.download(["AAPL", "GOOGL"], period="1d", interval="1h")

[*********************100%***********************]  2 of 2 completed


In [14]:
data

Price                           Close                    High              \
Ticker                           AAPL       GOOGL        AAPL       GOOGL   
Datetime                                                                    
2025-01-24 14:30:00+00:00  222.729996  198.820099  225.630005  200.250000   
2025-01-24 15:30:00+00:00  222.759995  199.399994  223.369995  199.500000   
2025-01-24 16:30:00+00:00  222.279999  199.289993  223.490005  199.690002   
2025-01-24 17:30:00+00:00  222.160004  199.899994  222.724503  200.110001   
2025-01-24 18:30:00+00:00  222.369995  200.721100  222.550003  200.899994   
2025-01-24 19:30:00+00:00  222.320007  199.899994  222.929993  200.779999   
2025-01-24 20:30:00+00:00  222.789993  200.199997  223.044998  200.289993   

Price                             Low                    Open              \
Ticker                           AAPL       GOOGL        AAPL       GOOGL   
Datetime                                                                    
2025-01-24 14:30:00+00:00  222.530106  198.000000  224.975006  198.169998   
2025-01-24 15:30:00+00:00  222.350098  198.309998  222.735001  198.830002   
2025-01-24 16:30:00+00:00  222.139999  198.789993  222.770004  199.419998   
2025-01-24 17:30:00+00:00  221.949997  199.210007  222.279999  199.259995   
2025-01-24 18:30:00+00:00  221.919998  199.729996  222.139999  199.910004   
2025-01-24 19:30:00+00:00  221.410004  199.493195  222.350006  200.690002   
2025-01-24 20:30:00+00:00  222.199997  199.770004  222.315002  199.889999   

Price                        Volume           
Ticker                         AAPL    GOOGL  
Datetime                                      
2025-01-24 14:30:00+00:00  15246196  4544808  
2025-01-24 15:30:00+00:00   6090500  1810501  
2025-01-24 16:30:00+00:00   5067407  2078771  
2025-01-24 17:30:00+00:00   3586687  2210418  
2025-01-24 18:30:00+00:00   2912495  2662566  
2025-01-24 19:30:00+00:00   4686564  2574820  
2025-01-24 20:30:00+00:00   5237044  3245992

In [15]:
def fetch_hourly_ticker_data(symbols: list) -> pd.DataFrame:
    data = yf.download(symbols, period="1d", interval="1h")
    
    df_flattened = pd.DataFrame()

    for ticker in data.columns.get_level_values(1).unique():
        ticker_df = data.xs(ticker, axis=1, level=1)
        ticker_df["Symbol"] = ticker
        df_flattened = pd.concat([df_flattened, ticker_df])

    df_flattened.reset_index(inplace=True)
    df_flattened.rename(columns={"Datetime": "Timestamp"}, inplace=True)
    df_flattened.columns.name = None
    return df_flattened

In [16]:
test = fetch_hourly_ticker_data(["AAPL", "GOOGL"])
test

[*********************100%***********************]  2 of 2 completed
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker


,Timestamp,Close,High,Low,Open,Volume,Symbol
0,2025-01-24 14:30:00+00:00,222.729996,225.630005,222.530106,224.975006,15246196,AAPL
1,2025-01-24 15:30:00+00:00,222.759995,223.369995,222.350098,222.735001,6090500,AAPL
2,2025-01-24 16:30:00+00:00,222.279999,223.490005,222.139999,222.770004,5067407,AAPL
3,2025-01-24 17:30:00+00:00,222.160004,222.724503,221.949997,222.279999,3586687,AAPL
4,2025-01-24 18:30:00+00:00,222.369995,222.550003,221.919998,222.139999,2912495,AAPL
5,2025-01-24 19:30:00+00:00,222.320007,222.929993,221.410004,222.350006,4686564,AAPL
6,2025-01-24 20:30:00+00:00,222.789993,223.044998,222.199997,222.315002,5237044,AAPL
7,2025-01-24 14:30:00+00:00,198.820099,200.250000,198.000000,198.169998,4544808,GOOGL
8,2025-01-24 15:30:00+00:00,199.399994,199.500000,198.309998,198.830002,1810501,GOOGL
9,2025-01-24 16:30:00+00:00,199.289993,199.690002,198.789993,199.419998,2078771,GOOGL


In [17]:
last_fetched = None

def filter_relevant_data(symbols: list) -> pd.DataFrame:
    global last_fetched
    data = fetch_hourly_ticker_data(symbols)
    if last_fetched:
        data = data[data["Timestamp"] > last_fetched]

    if not data.empty:
        last_fetched = data["Timestamp"].max()
        return data

    return pd.DataFrame()

In [18]:
last_fetched = None
lmao = filter_relevant_data(["AAPL", "GOOGL"])
lmao.shape

[*********************100%***********************]  2 of 2 completed
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker


(14, 7)

In [19]:
import time

last_fetched = None

while True:
    data = filter_relevant_data(["AAPL", "MSFT", "GOOGL"])
    print(f"Fetched {data.shape[0]} rows")
    time.sleep(60)

[*********************100%***********************]  3 of 3 completed
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df["Symbol"] = ticker
/var/folders/3s/jxqtlgkn21q5w9l6jfks93g00000gn/T/ipykernel_27061/2802342868.py:8: SettingWithCopyWarning: 
A value is trying to b

Fetched 21 rows


KeyboardInterrupt: 